In [203]:
import pandas as pd
from IPython.display import display

In [204]:
matches = pd.read_csv("matches.csv", index_col=0)
matches.head()

,date,time,comp,round,day,venue,result,gf,ga,opponent,...,tklw,blocks,int,clr,touches,att_y,succ,succ%,season,team
1,2023-08-19,15:30,Bundesliga,Matchweek 1,Sat,Home,W,3.0,2.0,RB Leipzig,...,11.0,14.0,8.0,15.0,574.0,22.0,11.0,50.0,2023-2024,Bayer Leverkusen
2,2023-08-26,18:30,Bundesliga,Matchweek 2,Sat,Away,W,3.0,0.0,M'Gladbach,...,11.0,9.0,9.0,11.0,854.0,16.0,7.0,43.8,2023-2024,Bayer Leverkusen
3,2023-09-02,15:30,Bundesliga,Matchweek 3,Sat,Home,W,5.0,1.0,Darmstadt 98,...,4.0,10.0,4.0,3.0,913.0,26.0,15.0,57.7,2023-2024,Bayer Leverkusen
4,2023-09-15,20:30,Bundesliga,Matchweek 4,Fri,Away,D,2.0,2.0,Bayern Munich,...,12.0,9.0,21.0,19.0,744.0,33.0,16.0,48.5,2023-2024,Bayer Leverkusen
6,2023-09-24,15:30,Bundesliga,Matchweek 5,Sun,Home,W,4.0,1.0,Heidenheim,...,10.0,11.0,4.0,12.0,964.0,23.0,14.0,60.9,2023-2024,Bayer Leverkusen


# Data preprocessing

## Get to know data

In [205]:
# drop unnecessary columns
matches_dropped = matches.drop(columns=['time', 'day', 'attendance', 'captain', 'formation', 'referee',
                                        'match report', 'notes'])
matches_dropped.head()

,date,comp,round,venue,result,gf,ga,opponent,xg,xga,...,tklw,blocks,int,clr,touches,att_y,succ,succ%,season,team
1,2023-08-19,Bundesliga,Matchweek 1,Home,W,3.0,2.0,RB Leipzig,1.4,1.6,...,11.0,14.0,8.0,15.0,574.0,22.0,11.0,50.0,2023-2024,Bayer Leverkusen
2,2023-08-26,Bundesliga,Matchweek 2,Away,W,3.0,0.0,M'Gladbach,2.7,0.9,...,11.0,9.0,9.0,11.0,854.0,16.0,7.0,43.8,2023-2024,Bayer Leverkusen
3,2023-09-02,Bundesliga,Matchweek 3,Home,W,5.0,1.0,Darmstadt 98,2.8,0.4,...,4.0,10.0,4.0,3.0,913.0,26.0,15.0,57.7,2023-2024,Bayer Leverkusen
4,2023-09-15,Bundesliga,Matchweek 4,Away,D,2.0,2.0,Bayern Munich,2.1,2.1,...,12.0,9.0,21.0,19.0,744.0,33.0,16.0,48.5,2023-2024,Bayer Leverkusen
6,2023-09-24,Bundesliga,Matchweek 5,Home,W,4.0,1.0,Heidenheim,3.4,0.5,...,10.0,11.0,4.0,12.0,964.0,23.0,14.0,60.9,2023-2024,Bayer Leverkusen


Description:

Shooting:
- Sh = shots total
- SoT = shots on target
- SoT% = shots on target %
- Touches = number of times a player touches the ball

Possession:
- Att = take-ons attempted
- Succ = succesful take-ons
- Succ% = succesful take-ons %

In [206]:
# shape
matches_dropped.shape

(2826, 39)

In [207]:
# take a look at first info 
matches_dropped.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2826 entries, 1 to 35
Data columns (total 39 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   date      2826 non-null   object 
 1   comp      2826 non-null   object 
 2   round     2826 non-null   object 
 3   venue     2826 non-null   object 
 4   result    2826 non-null   object 
 5   gf        2826 non-null   float64
 6   ga        2826 non-null   float64
 7   opponent  2826 non-null   object 
 8   xg        2824 non-null   float64
 9   xga       2824 non-null   float64
 10  poss      2826 non-null   float64
 11  sh        2826 non-null   float64
 12  sot       2826 non-null   float64
 13  sot%      2825 non-null   float64
 14  saves     2826 non-null   float64
 15  save%     2757 non-null   float64
 16  stp       2824 non-null   float64
 17  stp%      2824 non-null   float64
 18  #opa      2824 non-null   float64
 19  cmp       2824 non-null   float64
 20  att_x     2824 non-null   float64
 21

In [208]:
# take a look at the columns
matches_dropped.columns

Index(['date', 'comp', 'round', 'venue', 'result', 'gf', 'ga', 'opponent',
       'xg', 'xga', 'poss', 'sh', 'sot', 'sot%', 'saves', 'save%', 'stp',
       'stp%', '#opa', 'cmp', 'att_x', 'cmp%', 'totdist', 'prgdist', 'ast',
       'kp', 'ppa', 'crspa', 'tkl', 'tklw', 'blocks', 'int', 'clr', 'touches',
       'att_y', 'succ', 'succ%', 'season', 'team'],
      dtype='object')

In [209]:
# check dtypes
matches_dropped.dtypes

date         object
comp         object
round        object
venue        object
result       object
gf          float64
ga          float64
opponent     object
xg          float64
xga         float64
poss        float64
sh          float64
sot         float64
sot%        float64
saves       float64
save%       float64
stp         float64
stp%        float64
#opa        float64
cmp         float64
att_x       float64
cmp%        float64
totdist     float64
prgdist     float64
ast           int64
kp          float64
ppa         float64
crspa       float64
tkl         float64
tklw        float64
blocks      float64
int         float64
clr         float64
touches     float64
att_y       float64
succ        float64
succ%       float64
season       object
team         object
dtype: object

In [210]:
# some statistics summary
matches_dropped.describe()

,gf,ga,xg,xga,poss,sh,sot,sot%,saves,save%,...,crspa,tkl,tklw,blocks,int,clr,touches,att_y,succ,succ%
count,2826.000000,2826.000000,2824.000000,2824.000000,2826.000000,2826.000000,2826.000000,2825.000000,2826.000000,2757.000000,...,2824.000000,2824.000000,2826.000000,2824.000000,2826.000000,2824.000000,2824.000000,2824.000000,2824.000000,2824.000000
mean,1.567587,1.567587,1.478470,1.478470,50.000354,12.788747,4.426752,34.747221,3.029016,67.878527,...,2.113314,16.054887,9.554494,11.680595,10.667728,19.361544,615.812323,16.781161,8.875354,52.579533
std,1.350921,1.350921,0.855113,0.855113,11.126386,5.169717,2.546234,15.584948,2.038533,26.870918,...,1.763540,5.135310,3.594889,3.996274,4.343849,8.715438,122.447037,6.510156,4.391815,15.715098
min,0.000000,0.000000,0.000000,0.000000,19.000000,0.000000,0.000000,0.000000,0.000000,-100.000000,...,0.000000,3.000000,0.000000,1.000000,0.000000,0.000000,321.000000,3.000000,0.000000,0.000000
25%,1.000000,1.000000,0.900000,0.900000,42.000000,9.000000,3.000000,25.000000,1.000000,50.000000,...,1.000000,12.000000,7.000000,9.000000,8.000000,13.000000,526.000000,12.000000,6.000000,42.100000
50%,1.000000,1.000000,1.300000,1.300000,50.000000,12.000000,4.000000,33.300000,3.000000,70.000000,...,2.000000,16.000000,9.000000,11.000000,10.000000,18.000000,603.000000,16.000000,8.000000,53.300000
75%,2.000000,2.000000,1.900000,1.900000,58.000000,16.000000,6.000000,44.400000,4.000000,88.900000,...,3.000000,19.000000,12.000000,14.000000,13.000000,24.000000,692.000000,21.000000,11.000000,63.200000
max,8.000000,8.000000,5.800000,5.800000,81.000000,34.000000,20.000000,100.000000,19.000000,100.000000,...,13.000000,34.000000,24.000000,28.000000,29.000000,62.000000,1101.000000,46.000000,28.000000,100.000000


## Data cleaning

### Duplicates

In [211]:
matches_dropped.head()

,date,comp,round,venue,result,gf,ga,opponent,xg,xga,...,tklw,blocks,int,clr,touches,att_y,succ,succ%,season,team
1,2023-08-19,Bundesliga,Matchweek 1,Home,W,3.0,2.0,RB Leipzig,1.4,1.6,...,11.0,14.0,8.0,15.0,574.0,22.0,11.0,50.0,2023-2024,Bayer Leverkusen
2,2023-08-26,Bundesliga,Matchweek 2,Away,W,3.0,0.0,M'Gladbach,2.7,0.9,...,11.0,9.0,9.0,11.0,854.0,16.0,7.0,43.8,2023-2024,Bayer Leverkusen
3,2023-09-02,Bundesliga,Matchweek 3,Home,W,5.0,1.0,Darmstadt 98,2.8,0.4,...,4.0,10.0,4.0,3.0,913.0,26.0,15.0,57.7,2023-2024,Bayer Leverkusen
4,2023-09-15,Bundesliga,Matchweek 4,Away,D,2.0,2.0,Bayern Munich,2.1,2.1,...,12.0,9.0,21.0,19.0,744.0,33.0,16.0,48.5,2023-2024,Bayer Leverkusen
6,2023-09-24,Bundesliga,Matchweek 5,Home,W,4.0,1.0,Heidenheim,3.4,0.5,...,10.0,11.0,4.0,12.0,964.0,23.0,14.0,60.9,2023-2024,Bayer Leverkusen


In [212]:
# numeric and non-numeric columns separate
numeric_col = matches_dropped.select_dtypes(include='number').columns
non_numeric_col = matches_dropped.select_dtypes(exclude='number').columns
display(numeric_col)
display(non_numeric_col)

Index(['gf', 'ga', 'xg', 'xga', 'poss', 'sh', 'sot', 'sot%', 'saves', 'save%',
       'stp', 'stp%', '#opa', 'cmp', 'att_x', 'cmp%', 'totdist', 'prgdist',
       'ast', 'kp', 'ppa', 'crspa', 'tkl', 'tklw', 'blocks', 'int', 'clr',
       'touches', 'att_y', 'succ', 'succ%'],
      dtype='object')

Index(['date', 'comp', 'round', 'venue', 'result', 'opponent', 'season',
       'team'],
      dtype='object')

In [213]:
# list all duplicated rows
matches_dropped.loc[matches_dropped.duplicated(keep='first'),:]

,date,comp,round,venue,result,gf,ga,opponent,xg,xga,...,tklw,blocks,int,clr,touches,att_y,succ,succ%,season,team


### Missing values

In [214]:
matches_dropped.shape

(2826, 39)

In [215]:
# list the number of missing values in each column
matches_dropped.isnull().sum().sort_values(ascending=False)

save%       69
cmp          2
kp           2
prgdist      2
totdist      2
cmp%         2
att_x        2
crspa        2
#opa         2
stp%         2
stp          2
tkl          2
ppa          2
blocks       2
xga          2
xg           2
clr          2
touches      2
att_y        2
succ         2
succ%        2
sot%         1
int          0
season       0
tklw         0
date         0
ast          0
comp         0
saves        0
sot          0
sh           0
poss         0
opponent     0
ga           0
gf           0
result       0
venue        0
round        0
team         0
dtype: int64

In [216]:
# percentage of missing values
matches_dropped.isnull().mean().sort_values(ascending=False)*100

save%       2.441614
cmp         0.070771
kp          0.070771
prgdist     0.070771
totdist     0.070771
cmp%        0.070771
att_x       0.070771
crspa       0.070771
#opa        0.070771
stp%        0.070771
stp         0.070771
tkl         0.070771
ppa         0.070771
blocks      0.070771
xga         0.070771
xg          0.070771
clr         0.070771
touches     0.070771
att_y       0.070771
succ        0.070771
succ%       0.070771
sot%        0.035386
int         0.000000
season      0.000000
tklw        0.000000
date        0.000000
ast         0.000000
comp        0.000000
saves       0.000000
sot         0.000000
sh          0.000000
poss        0.000000
opponent    0.000000
ga          0.000000
gf          0.000000
result      0.000000
venue       0.000000
round       0.000000
team        0.000000
dtype: float64

**sot%**

In [217]:
# list all rows that sot% is null
matches_dropped.loc[matches_dropped.loc[:,'sot%'].isnull(),:]

,date,comp,round,venue,result,gf,ga,opponent,xg,xga,...,tklw,blocks,int,clr,touches,att_y,succ,succ%,season,team
8,2020-11-21,Bundesliga,Matchweek 8,Home,L,1.0,2.0,Leverkusen,0.0,1.2,...,12.0,8.0,20.0,38.0,489.0,3.0,0.0,0.0,2020-2021,Arminia


In [218]:
# find reason
matches_dropped.loc[matches_dropped.loc[:,'sot%'].isnull(),:][['sh','sot','sot%']]

,sh,sot,sot%
8,0.0,0.0,NaN


In [219]:
# fill missing values with 0
matches_dropped.loc[:,'sot%'].fillna(0, inplace=True)

In [220]:
# check
matches_dropped.loc[matches_dropped.loc[:,'sot%'].isnull(),:]

,date,comp,round,venue,result,gf,ga,opponent,xg,xga,...,tklw,blocks,int,clr,touches,att_y,succ,succ%,season,team


**suc** and **suc%**

In [221]:
# find rows
matches_dropped.loc[matches_dropped.loc[:,'succ'].isnull(),:]

,date,comp,round,venue,result,gf,ga,opponent,xg,xga,...,tklw,blocks,int,clr,touches,att_y,succ,succ%,season,team
29,2022-03-18,Bundesliga,Matchweek 27,Away,W,2.0,0.0,Bochum,NaN,NaN,...,8.0,NaN,2.0,NaN,NaN,NaN,NaN,NaN,2021-2022,Monchengladbach
30,2022-03-18,Bundesliga,Matchweek 27,Home,L,0.0,2.0,M'Gladbach,NaN,NaN,...,9.0,NaN,9.0,NaN,NaN,NaN,NaN,NaN,2021-2022,Bochum


In [223]:
matches_dropped.groupby(['season', 'team'])[['succ', 'succ%']].transform('mean')

,succ,succ%
1,11.190476,49.342857
2,11.190476,49.342857
3,11.190476,49.342857
4,11.190476,49.342857
6,11.190476,49.342857
...,...,...
31,11.235294,54.964706
32,11.235294,54.964706
33,11.235294,54.964706
34,11.235294,54.964706


In [224]:
a=matches_dropped.fillna(matches_dropped.groupby(['season', 'team'])[['succ', 'succ%']].transform('mean'))
a

,date,comp,round,venue,result,gf,ga,opponent,xg,xga,...,tklw,blocks,int,clr,touches,att_y,succ,succ%,season,team
1,2023-08-19,Bundesliga,Matchweek 1,Home,W,3.0,2.0,RB Leipzig,1.4,1.6,...,11.0,14.0,8.0,15.0,574.0,22.0,11.0,50.0,2023-2024,Bayer Leverkusen
2,2023-08-26,Bundesliga,Matchweek 2,Away,W,3.0,0.0,M'Gladbach,2.7,0.9,...,11.0,9.0,9.0,11.0,854.0,16.0,7.0,43.8,2023-2024,Bayer Leverkusen
3,2023-09-02,Bundesliga,Matchweek 3,Home,W,5.0,1.0,Darmstadt 98,2.8,0.4,...,4.0,10.0,4.0,3.0,913.0,26.0,15.0,57.7,2023-2024,Bayer Leverkusen
4,2023-09-15,Bundesliga,Matchweek 4,Away,D,2.0,2.0,Bayern Munich,2.1,2.1,...,12.0,9.0,21.0,19.0,744.0,33.0,16.0,48.5,2023-2024,Bayer Leverkusen
6,2023-09-24,Bundesliga,Matchweek 5,Home,W,4.0,1.0,Heidenheim,3.4,0.5,...,10.0,11.0,4.0,12.0,964.0,23.0,14.0,60.9,2023-2024,Bayer Leverkusen
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31,2020-06-06,Bundesliga,Matchweek 30,Away,D,1.0,1.0,RB Leipzig,1.1,1.4,...,8.0,18.0,15.0,18.0,588.0,15.0,12.0,80.0,2019-2020,Paderborn 07
32,2020-06-13,Bundesliga,Matchweek 31,Home,L,1.0,5.0,Werder Bremen,1.6,2.9,...,9.0,5.0,9.0,17.0,532.0,14.0,9.0,64.3,2019-2020,Paderborn 07
33,2020-06-16,Bundesliga,Matchweek 32,Away,L,0.0,1.0,Union Berlin,0.5,1.4,...,5.0,10.0,13.0,12.0,650.0,22.0,13.0,59.1,2019-2020,Paderborn 07
34,2020-06-20,Bundesliga,Matchweek 33,Home,L,1.0,3.0,M'Gladbach,0.7,3.5,...,8.0,9.0,4.0,12.0,578.0,22.0,9.0,40.9,2019-2020,Paderborn 07


In [227]:
a.loc[matches_dropped.loc[:,'succ'].isnull(),:]

,date,comp,round,venue,result,gf,ga,opponent,xg,xga,...,tklw,blocks,int,clr,touches,att_y,succ,succ%,season,team
29,2022-03-18,Bundesliga,Matchweek 27,Away,W,2.0,0.0,Bochum,NaN,NaN,...,8.0,NaN,2.0,NaN,NaN,NaN,10.878788,59.130303,2021-2022,Monchengladbach
30,2022-03-18,Bundesliga,Matchweek 27,Home,L,0.0,2.0,M'Gladbach,NaN,NaN,...,9.0,NaN,9.0,NaN,NaN,NaN,6.848485,51.096970,2021-2022,Bochum


In [226]:
a.loc[a.loc[:,'succ'].isnull(),:]

,date,comp,round,venue,result,gf,ga,opponent,xg,xga,...,tklw,blocks,int,clr,touches,att_y,succ,succ%,season,team


In [ ]:
# conditions to filter season and team
conditions_Mochengladbach = (matches_dropped.loc[:,'season']=="2021-2022")&(matches_dropped.loc[:,'team']=="Monchengladbach")
# mean succ of Monchengladbach
Monchengladbach_succ_mean = matches_dropped.loc[conditions_Mochengladbach,'succ'].mean()
# mean succ of Monchengladbach
Monchengladbach_succPercent_mean = matches_dropped.loc[conditions_Mochengladbach,'succ%'].mean()

In [ ]:
Monchengladbach_succ_mean

In [ ]:
Monchengladbach_succPercent_mean

In [ ]:
matches_dropped.loc[conditions_Mochengladbach, 'succ'].isnull().sum()

In [ ]:
# replace by mean value in this season 2021-2022
matches_dropped.loc[conditions_Mochengladbach, 'succ'].fillna(Monchengladbach_succ_mean, inplace=True)

In [ ]:
matches_dropped.loc[conditions_Mochengladbach, 'succ']

In [ ]:
# Calculate means of 'succ' and 'succ%' for each team in the '2021-2022' season
means_2021_2022 = matches_dropped.loc[(matches_dropped['season'] == '2021-2022') & 
                                  (matches_dropped['team'].isin(['Mochengladbach', 'Bochum'])), ['succ', 'succ%']].mean()
means_2021_2022

In [ ]:
matches_dropped.loc[(matches_dropped['season'] == '2021-2022') & 
                    (matches_dropped['team'].isin(['Mochengladbach', 'Bochum'])), 
                    ['succ', 'succ%']] = means_2021_2022['succ'], means_2021_2022['succ%']

In [ ]:
# Calculate means of 'succ' and 'succ%' for each team in the '2021-2022' season
means_2021_2022 = matches_dropped[(matches_dropped['season'] == '2021-2022') & 
                                  (matches_dropped['team'].isin(['Mochengladbach', 'Bochum']))].mean()

# Replace NaN values in 'succ' and 'succ%' for the specified conditions
matches_dropped.loc[(matches_dropped['season'] == '2021-2022') & 
                    (matches_dropped['team'].isin(['Mochengladbach', 'Bochum'])), 
                    ['succ', 'succ%']] = means_2021_2022['succ'], means_2021_2022['succ%']

# Now 'matches_dropped' contains the DataFrame with NaN values filled with means for the specified conditions


In [ ]:
# Group by 'team' and 'season' and calculate means of 'succ' and 'succ%'
means = matches_dropped.groupby(['team', 'season']).agg({'succ': 'mean', 'succ%': 'mean'}).reset_index()

# Filter for the '2021-2022' season
means_2021_2022 = means[means['season'] == '2021-2022']

# Merge means back into the original DataFrame to fill NaN values
filled_df = pd.merge(matches_dropped, means_2021_2022, on=['team', 'season'], suffixes=('', '_mean'))

# Replace NaN values in 'succ' and 'succ%' with the corresponding means
filled_df['succ'].fillna(filled_df['succ_mean'], inplace=True)
filled_df['succ%'].fillna(filled_df['succ%_mean'], inplace=True)

# Drop the redundant columns
filled_df.drop(columns=['succ_mean', 'succ%_mean'], inplace=True)

# Now 'filled_df' contains the DataFrame with NaN values filled with means for each team in the '2021-2022' season


In [ ]:
test = matches_dropped.loc[conditions_Mochengladbach, 'succ']
test

In [ ]:
type(test)

In [ ]:
test.fillna(Monchengladbach_succ_mean, inplace=True)

In [ ]:
test

In [ ]:
test.isnull().sum()

In [ ]:
matches_dropped.loc[matches_dropped.loc[:,'team']=='Monchengladbach' ,'succ'].isnull().sum()

In [ ]:
# Check the mean value calculated for Monchengladbach's 'succ' column in 2021-2022 season
print("Mean succ for Monchengladbach in 2021-2022 season:", Monchengladbach_succ_mean)

# Check if there are still missing values in the 'succ' column for Monchengladbach
print("Number of missing values for Monchengladbach:", matches_dropped.loc[(matches_dropped['team'] == 'Monchengladbach') & (matches_dropped['season'] == '2021-2022'), 'succ'].isnull().sum())


In [ ]:
# Find the row with the missing value for Monchengladbach in 2021-2022 season
missing_row = matches_dropped.loc[(matches_dropped['team'] == 'Monchengladbach') & (matches_dropped['season'] == '2021-2022') & matches_dropped['succ'].isnull()]

missing_row


In [ ]:
matches_dropped.loc[matches_dropped.loc[:,'succ'].isnull(),:]

# EDA

# Feature engineering

In [ ]:
# turn date column to datetime
# matches['date'] = pd.to_datetime(matches['date'])